# ***About the dataset***

**id**: unique id for a news article

**title**: the title of a news article

**author**: author of a news article

**text**: the text of the article; could be incomplete

**label**: a label that marks whether the news article is fake or not

In [ ]:
#importing the dependies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printing the english words
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data - Pre proccessing

In [ ]:
# loading the dataset into pandas dataframe
news_dataset = pd.read_csv('/content/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
#print first 5 rows of dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#Counting the missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# Replace the null value with empty set
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the text and author into content
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['text']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn Ever get the feeling your life...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss Videos 15 Civilians Killed In ...
4        Howard Portnoy Print \nAn Iranian woman has be...
                               ...                        
20795    Jerome Hudson Rapper T. I. unloaded on black c...
20796    Benjamin Hoffman When the Green Bay Packers lo...
20797    Michael J. de la Merced and Rachel Abrams The ...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson   David Swanson is an author, ac...
Name: content, Length: 20800, dtype: object


In [ ]:
# seperating the data and label
X = news_dataset.drop('label',axis=1)
Y = news_dataset['label']

In [ ]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Stemming : Stemming is a process of reducing a word to its root word 

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('^a-zA-Z',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn ever get feel life circl round...
2        consortiumnews.com truth might get fire octob ...
3        jessica purkiss video 15 civilian kill singl u...
4        howard portnoy print iranian woman sentenc six...
                               ...                        
20795    jerom hudson rapper t. i. unload black celebr ...
20796    benjamin hoffman green bay packer lost washing...
20797    michael j. de la merc rachel abram macy’ today...
20798    alex ansari nato, russia hold parallel exercis...
20799    david swanson david swanson author, activist, ...
Name: content, Length: 20800, dtype: object


In [ ]:
# seperating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values


In [ ]:
print(X)

['darrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet darrel lucu octob 30, 2016 subscrib jason chaffetz stump american fork, utah ( imag courtesi michael jolley, avail creativ commons-bi license) apolog keith olbermann, doubt worst person world week–fbi director jame comey. accord hous democrat aide, look like also know second-worst person well. turn comey sent now-infam letter announc fbi look email may relat hillari clinton’ email server, rank democrat relev committe didn’t hear comey. found via tweet one republican committe chairmen. know, comey notifi republican chairmen democrat rank member hous intelligence, judiciary, oversight committe agenc review email recent discov order see contain classifi information. long letter went out, oversight committe chairman jason chaffetz set polit world ablaz tweet. fbi dir inform me, "the fbi learn exist email appear pertin investigation." case reopen — jason chaffetz (@jasoninthehouse) octob 28, 2016 course, know ca

In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

TfidfVectorizer()

In [ ]:
X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 151428)	0.050071913039277015
  (0, 151375)	0.015220356355382745
  (0, 151364)	0.010293533329965033
  (0, 150288)	0.04506853036683926
  (0, 150282)	0.09672892464338974
  (0, 150212)	0.03855363264055311
  (0, 150154)	0.011679644191508751
  (0, 149484)	0.032144278629615004
  (0, 148663)	0.024772456936669907
  (0, 148296)	0.01743360932321059
  (0, 148189)	0.012807802735056963
  (0, 148077)	0.029820985596897383
  (0, 147974)	0.013487481211466602
  (0, 147805)	0.012332528734804756
  (0, 147726)	0.012165129891091144
  (0, 146599)	0.026518597367161906
  (0, 146554)	0.0322523572978056
  (0, 145299)	0.021965872818272617
  (0, 143550)	0.06666185078693142
  (0, 142575)	0.03413699178360405
  (0, 142182)	0.01783355345603463
  (0, 140998)	0.03936304561109675
  (0, 140209)	0.011298859431651901
  (0, 140195)	0.044040664811357426
  (0, 140113)	0.11937983509258049
  :	:
  (20799, 6793)	0.03147682893707254
  (20799, 6711)	0.03259570613556846
  (20799, 6338)	0.015215116987174662
  (20799, 5515)	0.018

Splitting the dataset into training and testing data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the model : Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation and Accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [ ]:
print(training_data_accuracy)

0.9814302884615385


In [ ]:
# accuracy score on the testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print(testing_data_accuracy)

0.9552884615384616


Making a predictive system

In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')  

[1]
The news is Fake
